# Security


* This notebook uses Application Default Credentials (ADC) for BigQuery authentication.
* Authenticate with gcloud before running: `gcloud auth application-default login`
* No API key files needed - authentication is handled through your gcloud session.


# Tools

In [27]:
# Package and library installation
packages_needed = c("tidyverse", "knitr") # comma delimited vector of package names
packages_installed = packages_needed %in% rownames(installed.packages())

if (any(! packages_installed))
  install.packages(packages_needed[! packages_installed])
for (i in 1:length(packages_needed)) {
  library(packages_needed[i], character.only = T)
}

* Using Application Default Credentials - no manual key file needed if you've run `gcloud auth application-default login`


In [28]:
install.packages("bigrquery")
library(bigrquery)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



# Source

## Database Connection

In [29]:
# BigQuery Authentication using Application Default Credentials
# This will use your gcloud authentication automatically
# Run this first in terminal: gcloud auth application-default login
bq_auth()

In [30]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [31]:
billing <- bq_test_project()

## Database Query

### gridVeg_plant_intercepts

In [32]:
# vegetation_gridVeg_summaries:gridVeg_plant_intercepts
sql_plant_intercepts <-
"
  SELECT
    survey_ID, grid_point, year, key_plant_species
  FROM
    `mpg-data-warehouse.vegetation_gridVeg_summaries.gridVeg_foliar_cover_all`
"
bq_plant_intercepts <- bq_project_query(billing, sql_plant_intercepts)
tb_plant_intercepts <- bq_table_download(bq_plant_intercepts)

In [33]:
df_plant_intercepts <- as.data.frame(tb_plant_intercepts) %>%
  mutate(detection_type = "point_intercept") %>% glimpse()

Rows: 28,083
Columns: 5
$ survey_ID         <chr> "436", "436", "436", "436", "436", "436", "436", "43…
$ grid_point        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ year              <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011…
$ key_plant_species <int> 5, 481, 201, 554, 241, 16, 250, 102, 267, 154, 57, 2…
$ detection_type    <chr> "point_intercept", "point_intercept", "point_interce…


In [34]:
df_plant_intercepts %>% filter(is.na(key_plant_species))

survey_ID,grid_point,year,key_plant_species,detection_type
<chr>,<int>,<int>,<int>,<chr>


### gridVeg_additional_species

In [35]:
# vegetation_point_intercept_gridVeg:gridVeg_additional_species
sql_add_spec <-
  "
    SELECT
      survey_ID, grid_point, year, key_plant_species
    FROM
      `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_additional_species`
  "
bq_add_spec <- bq_project_query(billing, sql_add_spec)
tb_add_spec <- bq_table_download(bq_add_spec)

In [36]:
df_add_spec <- as.data.frame(tb_add_spec) %>%
  mutate(detection_type = "supplemental_obs") %>%
  filter(!is.na(key_plant_species)) %>%
  glimpse()

Rows: 14,038
Columns: 5
$ survey_ID         <chr> "308", "308", "308", "308", "308", "308", "308", "30…
$ grid_point        <int> 324, 324, 324, 324, 324, 324, 324, 324, 324, 324, 23…
$ year              <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011…
$ key_plant_species <int> 69, 72, 5, 82, 75, 174, 230, 67, 428, 529, 240, 308,…
$ detection_type    <chr> "supplemental_obs", "supplemental_obs", "supplementa…


In [37]:
df_add_spec %>% filter(is.na(key_plant_species))

survey_ID,grid_point,year,key_plant_species,detection_type
<chr>,<int>,<int>,<int>,<chr>


### gridVeg_survey_metadata


In [38]:
# vegetation_point_intercept_gridVeg:gridVeg_survey_metadata
sql_survey_meta <-
  "
    SELECT
      *
    FROM
      `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata`
  "
bq_survey_meta <- bq_project_query(billing, sql_survey_meta)
tb_survey_meta <- bq_table_download(bq_survey_meta)
df_survey_meta <- as.data.frame(tb_survey_meta) %>% glimpse()

Rows: 1,723
Columns: 6
$ survey_ID       <chr> "138", "139", "135", "134", "137", "136", "141", "143"…
$ grid_point      <int> 61, 60, 52, 51, 62, 44, 57, 66, 65, 59, 69, 76, 75, 68…
$ year            <int> 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, …
$ date            <date> 2010-08-27, 2010-08-27, 2010-08-27, 2010-08-27, 2010-…
$ survey_sequence <chr> "2010", "2010", "2010", "2010", "2010", "2010", "2010"…
$ surveyor        <chr> "EAR", "EAR", "EAR", "EAR", "EAR", "EAR", "EAR", "EAR"…


### location_position_classification

In [39]:
# grid_point_summaries:location_position_classification
sql_loc_pos <-
  "
    SELECT
      *
    FROM
      `mpg-data-warehouse.grid_point_summaries.location_position_classification`
  "
bq_loc_pos <- bq_project_query(billing, sql_loc_pos)
tb_loc_pos <- bq_table_download(bq_loc_pos)
df_loc_pos <- as.data.frame(tb_loc_pos) %>% glimpse()

Rows: 588
Columns: 16
$ grid_point                  <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…
$ lat                         <dbl> 46.73193, 46.72972, 46.72443, 46.72487, 46…
$ long                        <dbl> -114.0017, -114.0010, -114.0227, -114.0195…
$ aspect_mean_deg             <dbl> 321.18675, 65.92522, 240.01216, 290.66936,…
$ aspect_direction            <chr> "NW", "ENE", "WSW", "WNW", "W", "SW", "W",…
$ aspect_northness            <dbl> 0.77919303, 0.40792854, -0.49981612, 0.352…
$ aspect_eastness             <dbl> -0.62678403, 0.91301386, -0.86613154, -0.9…
$ elevation_mean_m            <dbl> 1395.468, 1455.145, 1126.677, 1164.549, 11…
$ slope_mean_deg              <dbl> 28.139095, 20.516106, 5.503165, 6.102558, …
$ cover_type_2016_gridVeg     <chr> "woodland/forest", "non-irrigated grasslan…
$ biomass_habitat_type        <chr> NA, "Range", "Range", "Range", "Range", "R…
$ type1_biome                 <chr> "forest", "rangeland", "rangeland", "range…
$ type2_vegetation

### vegetation_species_metadata

In [40]:
# vegetation_species_metadata:vegetation_species_metadata
sql_species_meta <-
  "
    SELECT
      *
    FROM
      `mpg-data-warehouse.vegetation_species_metadata.vegetation_species_metadata`
  "
bq_species_meta <- bq_project_query(billing, sql_species_meta)
tb_species_meta <- bq_table_download(bq_species_meta)
df_species_meta <- as.data.frame(tb_species_meta) %>% glimpse()

Rows: 769
Columns: 9
$ key_plant_species   <int> 656, 285, 21, 22, 130, 131, 129, 635, 659, 685, 74…
$ key_plant_code      <chr> "DYSBOT", "KOCSCO", "AMABLI", "AMARET", "CHEFRE", …
$ plant_name_sci      <chr> "Dysphania botrys", "Kochia scoparia", "Amaranthus…
$ plant_name_syn      <chr> "Chenopodium botrys", "Bassia scoparia", NA, NA, N…
$ plant_name_common   <chr> "Jerusalem oak goosefoot", "common kochia", "mat p…
$ plant_name_family   <chr> "Amaranthaceae", "Amaranthaceae", "Amaranthaceae",…
$ plant_native_status <chr> "nonnative", "nonnative", "nonnative", "nonnative"…
$ plant_life_cycle    <chr> "unknown", "unknown", "unknown", "unknown", "unkno…
$ plant_life_form     <chr> "forb", "forb", "forb", "forb", "forb", "forb", "f…


# Wrangle

In [41]:
df_plant_intercepts %>% glimpse()

Rows: 28,083
Columns: 5
$ survey_ID         <chr> "436", "436", "436", "436", "436", "436", "436", "43…
$ grid_point        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ year              <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011…
$ key_plant_species <int> 5, 481, 201, 554, 241, 16, 250, 102, 267, 154, 57, 2…
$ detection_type    <chr> "point_intercept", "point_intercept", "point_interce…


In [42]:
df_add_spec %>% glimpse()

Rows: 14,038
Columns: 5
$ survey_ID         <chr> "308", "308", "308", "308", "308", "308", "308", "30…
$ grid_point        <int> 324, 324, 324, 324, 324, 324, 324, 324, 324, 324, 23…
$ year              <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011…
$ key_plant_species <int> 69, 72, 5, 82, 75, 174, 230, 67, 428, 529, 240, 308,…
$ detection_type    <chr> "supplemental_obs", "supplemental_obs", "supplementa…


## Bind rows

In [43]:
df_bind <- df_plant_intercepts %>%
  # Remove "NV" key plant code which is equal to 360
  filter(key_plant_species != 360) %>%
  bind_rows(df_add_spec) %>%
  glimpse()

Rows: 42,121
Columns: 5
$ survey_ID         <chr> "436", "436", "436", "436", "436", "436", "436", "43…
$ grid_point        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ year              <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011…
$ key_plant_species <int> 5, 481, 201, 554, 241, 16, 250, 102, 267, 154, 57, 2…
$ detection_type    <chr> "point_intercept", "point_intercept", "point_interce…


## Identify multiples

In [44]:
df_multiple <- df_bind %>%
  group_by(survey_ID, key_plant_species) %>%
  summarise(n = n()) %>%
  arrange(desc(n)) %>%
  filter(n > 1) %>%
  ungroup() %>%
  unite("multiple_key", survey_ID:key_plant_species, remove = FALSE) %>%
  glimpse()

`summarise()` has grouped output by 'survey_ID'. You can override using the
`.groups` argument.


Rows: 1,361
Columns: 4
$ multiple_key      <chr> "249_174", "326_57", "376_62", "425_250", "450_5", "…
$ survey_ID         <chr> "249", "326", "376", "425", "450", "483", "485", "CF…
$ key_plant_species <int> 174, 57, 62, 250, 5, 240, 5, 529, 132, 459, 435, 39,…
$ n                 <int> 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2…


In [45]:
# display multiple example
df_bind %>%
  filter(survey_ID == 249, key_plant_species == 174)

survey_ID,grid_point,year,key_plant_species,detection_type
<chr>,<int>,<int>,<int>,<chr>
249,67,2011,174,point_intercept
249,67,2011,174,supplemental_obs
249,67,2011,174,supplemental_obs


## Remove multiples

In [46]:
# use df_multiple to select and remove from df_supplemental
df_clean_supp <- df_add_spec %>%
  unite("multiple_key", c(survey_ID, key_plant_species), remove = FALSE) %>%
  anti_join(df_multiple, by = "multiple_key") %>%
  select(,-multiple_key) %>%
  glimpse()

Rows: 12,586
Columns: 5
$ survey_ID         <chr> "308", "308", "308", "308", "308", "308", "308", "30…
$ grid_point        <int> 324, 324, 324, 324, 324, 324, 324, 324, 324, 230, 23…
$ year              <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011…
$ key_plant_species <int> 69, 72, 5, 82, 75, 174, 230, 67, 529, 240, 308, 230,…
$ detection_type    <chr> "supplemental_obs", "supplemental_obs", "supplementa…


## Join clean dataframes

In [47]:
df_clean_supp %>%
  glimpse()

Rows: 12,586
Columns: 5
$ survey_ID         <chr> "308", "308", "308", "308", "308", "308", "308", "30…
$ grid_point        <int> 324, 324, 324, 324, 324, 324, 324, 324, 324, 230, 23…
$ year              <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011…
$ key_plant_species <int> 69, 72, 5, 82, 75, 174, 230, 67, 529, 240, 308, 230,…
$ detection_type    <chr> "supplemental_obs", "supplemental_obs", "supplementa…


In [48]:
df_clean_bind <- df_plant_intercepts %>%
  # Remove "NV" key plant code which is equal to 360
  filter(key_plant_species != 360) %>%
  bind_rows(df_clean_supp) %>%
  glimpse()

Rows: 40,669
Columns: 5
$ survey_ID         <chr> "436", "436", "436", "436", "436", "436", "436", "43…
$ grid_point        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ year              <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011…
$ key_plant_species <int> 5, 481, 201, 554, 241, 16, 250, 102, 267, 154, 57, 2…
$ detection_type    <chr> "point_intercept", "point_intercept", "point_interce…


In [49]:
# double check for multiples
df_clean_bind %>%
  group_by(survey_ID, key_plant_species) %>%
  summarise(n = n()) %>%
  arrange(desc(n)) %>%
  filter(n > 1) %>%
  # I'm not clear if I'm using ungroup in a meaningful way here
  ungroup() %>%
  glimpse()

`summarise()` has grouped output by 'survey_ID'. You can override using the
`.groups` argument.


Rows: 0
Columns: 3
$ survey_ID         <chr> 
$ key_plant_species <int> 
$ n                 <int> 


# Join in Additional Columns

## Filter

In [50]:
gridVeg_species_richness <- df_clean_bind %>%
  filter(year > 2022 & !is.na(key_plant_species)) %>%
  glimpse()
  

Rows: 2,597
Columns: 5
$ survey_ID         <chr> "27869B01-61AE-4DB3-A2AC-AD4D8C9A7ECD", "27869B01-61…
$ grid_point        <int> 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4…
$ year              <int> 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023…
$ key_plant_species <int> 529, 232, 320, 265, 80, 286, 153, 82, 411, 576, 437,…
$ detection_type    <chr> "point_intercept", "point_intercept", "point_interce…


# Output

In [51]:
# 2022-08-17 ES
# write_csv(gridVeg_species_richness, "gridVeg_species_richness-WRANGLE.csv")

# 2025-11-04 esamsoe
# write_csv(gridVeg_species_richness, "../data/processed/gridVeg_species_richness_WRANGLE-251104.csv")
